# Using the Materials API

We will cover:

* getting an API key
* `pymatgen.MPRester` built-ins
* `MPRester.query`
* the [mapidoc](https://github.com/materialsproject/mapidoc) repository
* accessing the API more directly, with plain HTTP requests (via the Python `requests` library)